## Tutorial 1: reading and preparing data for an HsM model

In this tutorial we will show you how to read data from :

1. raw (preprocessed) EEG data in *.bdf* or *.fif* format
2. epoched data in *.fif*
3. Data already transformed to PC space (e.g. previous applications of HsMM MVPA)

All these methods assume that the data has been preprocessed (e.g. see method section of [this paper](https://psyarxiv.com/nmg6w/)  for an example of preprocessing for HsMM-MVPA), if you don't know where to start for preprocessing EEG, the [MNE](https://mne.tools/stable/index.html) is full of tutorials on EEG data.

# 1. Reading raw data

For the following I give an example using (yet not public) preprocessed EEG data that has not yet been epoched.

First in order for the HsMM to run on each trial we have to declare the stimulus triggers as well as the response triggers which defines a trial

In [1]:
event_id = {'accuracy/left/1':211,'accuracy/left/2':212,'accuracy/left/3':213,
           'accuracy/right/1':221,'accuracy/right/2':222,'accuracy/right/3':223,
            'speed/left/1':111,'speed/left/2':112,'speed/left/3':113, #conditions in the experiment
           'speed/right/1':121,'speed/right/2':122,'speed/right/3':123} # used for segmentation
resp_id = {'r_left':100,'r_right':200}#Response events

Then we specify wich files we want to analyse, if for example you have all your participant dat inside one folder you can do the following :

In [2]:
import os
eeg_path = '../../../PHD/ForceEEG_2021/processing_EEG/preprocessed_data/'#Where the data is relative to where you run the code on your computer
subj_names = [x.split('_')[1].split('.')[0] for x in os.listdir(eeg_path) if 'preprocessed' in x]#select files which contain the string 'preprocessed'
subj_files = [eeg_path+ 'preprocessed_'+x+'.fif' for x in subj_names]#Create a list of files that can be read from the folder

Next we can directly use the ```subj_file``` list and feed it to the ```hsmm.utils.read_mne_EEG``` but first you have to decide on a number of parameters regarding EEG processing:

In [3]:
from hsmm_mvpy import utils
utils.read_mne_EEG?

ModuleNotFoundError: No module named 'hsmm_mvpy'

In [ ]:

sfreq = 100#For resampling



save_file = 'epoch_data_condition-wise_%s.nc'%version
if save_file not in os.listdir():#avoids to loop again if notebook is restarted
    hsmm.utils.read_mne_EEG(subj_files, event_id, resp_id, sfreq, tmin=-.25, tmax=2, offset_after_resp=.05, 
                            high_pass=.5, low_pass = 30, lower_limit_RT=.25, upper_limit_RT=2)
    epoch_data = xr.concat(epoch_data, pd.Series(subj_names, name="participant"),
                          fill_value={'event':'', 'data':np.nan})#not sure about the order of the particiapnt use with caution
    epoch_data.to_netcdf(save_file)
    epoch_data = xr.open_dataset(save_file)#For an unknown reason I need to re-read it
else:
    epoch_data = xr.open_dataset(save_file, engine='netcdf4')

raw = mne.io.read_raw_fif(subj_files[0], preload=False, verbose=False)#loading for sensor position
raw.set_montage(mne.channels.make_standard_montage('biosemi64'));